# **Recommendation System based on Link Prediction**

# PyG to SNAP

Load grpah from PyG and convert to SNAP for building a pipeline

## Loading Package

In [1]:
import torch
import torch_geometric
import torch.nn as nn
import torch.nn.functional as F
from torch_geometric.data import DataLoader
from deepsnap.graph import Graph
from deepsnap.batch import Batch
from deepsnap.dataset import GraphDataset
from torch_geometric.datasets import Planetoid, TUDataset
import networkx
import os

## Loading Graph

In [3]:
path = '/Users/marceloyou/Desktop/算法/RecommanationSystem/GNN(GCN)/data'
filename = 'graph_0612_20:36:56.pth'

graph = torch.load(os.path.join(path, filename))
print(graph)


{'graph': HeteroData(
  user={ num_nodes=61 },
  product={ x=[207, 1153] },
  (user, like, product)={ edge_index=[2, 207] }
)}


## EDA on graph

In [74]:
dataset = graph['graph']
print(f'Dataset: {dataset}:')
print('======================')
shape = dataset['product']['x'].shape
print(f'Number of features: {shape}')
num_node = dataset['user'].num_nodes
print(f'Number of Nodes: {num_node}')

Dataset: HeteroData(
  user={ num_nodes=61 },
  product={ x=[207, 1153] },
  (user, like, product)={ edge_index=[2, 207] }
):
Number of features: torch.Size([207, 1153])
Number of Nodes: 61


In [50]:
num = dataset['user','like','product'].edge_index.shape[1]
print(f'Number of Edges of the graph: {num}')

Number of Edges of the graph: 207


## Visulization

In [9]:
from torch_geometric.utils import to_networkx
G = to_networkx(dataset, to_undirected = True)

TypeError: 'HeteroData' object is not callable

## PyG to DeepSNAP

In [6]:
if 'IS_GRADESCOPE_ENV' not in os.environ:
  root = './tmp/cora'
  name = 'Cora'

  # The Cora dataset
  pyg_dataset= Planetoid(root, name)

  # PyG dataset to a list of deepsnap graphs
  graphs = GraphDataset.pyg_to_graphs(pyg_dataset)

  # Get the first deepsnap graph (CORA only has one graph)
  graph = graphs[0]
  print(graph)

Graph(G=[], edge_index=[2, 10556], edge_label_index=[2, 10556], node_feature=[2708, 1433], node_label=[2708], node_label_index=[2708])


### Build Networkx object

In [51]:
import networkx as nx
from networkx.algorithms.community import greedy_modularity_communities
import matplotlib.pyplot as plt
import copy


In [75]:
from torch_geometric.data import HeteroData
data = dataset.to_homogeneous()

In [87]:
data_nx =to_networkx(data, to_undirected = False)

In [77]:
len(list(data_nx.edges(data = True)))

124

In [12]:
label_mapping = {'user':0, 'product':1}
for i, node in enumerate(data_nx.nodes(data = True)):
    if i <= 61908:
        node[1]['node_label'] = 0
        node[1]['node_type'] ='user'
        node[1]['node_feature'] = torch.zeros(1153)
    else:
        node[1]['node_label'] = 1
        node[1]['node_feature'] = dataset['product'].x[i-61909]
        node[1]['node_type'] = 'product'
    

In [96]:
# small graph
label_mapping = {'user':0, 'product':1}
for i, node in enumerate(data_nx.nodes(data = True)):
    if i <= 60:
        node[1]['node_label'] = 0
        node[1]['node_type'] ='user'
        node[1]['node_feature'] = torch.zeros(1153)
    else:
        node[1]['node_label'] = 1
        node[1]['node_feature'] = dataset['product'].x[i-61]
        node[1]['node_type'] = 'product'
    

In [90]:
for u, v, edge in data_nx.edges(data = True):
    data_nx.add_edge(v, u)

In [104]:
list(data_nx.edges(data = True))[123:]

[(60, 162, {'edge_label': 0, 'edge_type': 'buy'}),
 (61, 0, {'edge_label': 1, 'edge_type': 'attract'}),
 (61, 3, {'edge_label': 1, 'edge_type': 'attract'}),
 (61, 5, {'edge_label': 1, 'edge_type': 'attract'}),
 (62, 1, {'edge_label': 1, 'edge_type': 'attract'}),
 (63, 2, {'edge_label': 1, 'edge_type': 'attract'}),
 (64, 2, {'edge_label': 1, 'edge_type': 'attract'}),
 (65, 4, {'edge_label': 1, 'edge_type': 'attract'}),
 (66, 6, {'edge_label': 1, 'edge_type': 'attract'}),
 (67, 7, {'edge_label': 1, 'edge_type': 'attract'}),
 (68, 2, {'edge_label': 1, 'edge_type': 'attract'}),
 (69, 2, {'edge_label': 1, 'edge_type': 'attract'}),
 (70, 3, {'edge_label': 1, 'edge_type': 'attract'}),
 (71, 8, {'edge_label': 1, 'edge_type': 'attract'}),
 (72, 2, {'edge_label': 1, 'edge_type': 'attract'}),
 (73, 9, {'edge_label': 1, 'edge_type': 'attract'}),
 (74, 10, {'edge_label': 1, 'edge_type': 'attract'}),
 (75, 11, {'edge_label': 1, 'edge_type': 'attract'}),
 (76, 9, {'edge_label': 1, 'edge_type': 'attra

In [94]:
for i, edge in enumerate(data_nx.edges(data = True)):
    if i <= 123:
        edge[2]['edge_label'] = 0
        edge[2]['edge_type'] = 'buy'
    else:
        edge[2]['edge_label'] = 1
        edge[2]['edge_type'] = 'attract'

In [99]:
from deepsnap.hetero_graph import HeteroGraph

data_snap = HeteroGraph(data_nx)

In [110]:
data.node_type[61909] # 前61908 is user

tensor(1)

In [105]:
data_snap.edge_index

{('user',
  'buy',
  'product'): tensor([[  0,   1,   1,   1,   1,   2,   2,   2,   2,   2,   2,   3,   3,   4,
            4,   5,   6,   6,   6,   6,   7,   8,   8,   9,   9,  10,  10,  10,
           10,  11,  11,  12,  12,  13,  13,  14,  14,  14,  15,  15,  16,  16,
           17,  17,  18,  18,  18,  18,  19,  20,  21,  21,  22,  23,  24,  25,
           25,  25,  26,  26,  27,  27,  28,  28,  28,  28,  29,  30,  30,  31,
           32,  33,  34,  35,  35,  35,  35,  35,  36,  36,  37,  37,  38,  38,
           38,  38,  38,  39,  40,  40,  41,  42,  43,  44,  45,  46,  47,  48,
           48,  48,  48,  48,  48,  48,  48,  48,  49,  50,  51,  52,  53,  54,
           54,  55,  56,  57,  57,  57,  57,  57,  57,  58,  59,  60],
         [  0,   1,  82,  83,  84,   2,   3,   7,   8,  11,  18,   0,   9,   4,
           60,   0,   5,  28,  72,  73,   6,  10,  47,  12,  15,  13,  35,  81,
           49,  14,  76,  16,  17,  19,  23,  20,  21,  22,  24,  25,  26,  99,
           27,  3

In [84]:
data_snap.edge_index[('user','buy','product')]

tensor([[  0,   1,   1,   1,   1,   2,   2,   2,   2,   2,   2,   3,   3,   4,
           4,   5,   6,   6,   6,   6,   7,   8,   8,   9,   9,  10,  10,  10,
          10,  11,  11,  12,  12,  13,  13,  14,  14,  14,  15,  15,  16,  16,
          17,  17,  18,  18,  18,  18,  19,  20,  21,  21,  22,  23,  24,  25,
          25,  25,  26,  26,  27,  27,  28,  28,  28,  28,  29,  30,  30,  31,
          32,  33,  34,  35,  35,  35,  35,  35,  36,  36,  37,  37,  38,  38,
          38,  38,  38,  39,  40,  40,  41,  42,  43,  44,  45,  46,  47,  48,
          48,  48,  48,  48,  48,  48,  48,  48,  49,  50,  51,  52,  53,  54,
          54,  55,  56,  57,  57,  57,  57,  57,  57,  58,  59,  60],
        [  0,   1,  82,  83,  84,   2,   3,   7,   8,  11,  18,   0,   9,   4,
          60,   0,   5,  28,  72,  73,   6,  10,  47,  12,  15,  13,  35,  81,
          49,  14,  76,  16,  17,  19,  23,  20,  21,  22,  24,  25,  26,  99,
          27,  34,  29,  36,  37,  58,  30,  31,  32,  33,  3

In [61]:
# check each component
data_snap.node_feature['user'].shape

torch.Size([61, 1153])

## EDA on DeepSnap Graph

### how many nodes of each type

In [106]:
num_node_0 = len(data_snap.node_type['product'])
num_node_1 = len(data_snap.node_type['user'])

print(f'The number of product in the data set is {num_node_0}')

print(f'The number of user in the data set is {num_node_1}')


The number of product in the data set is 207
The number of user in the data set is 61


### How many edges are of each message type

In [107]:
t = data_snap.message_types[0]
message_type_edges = [(t, len(data_snap.edge_type[t]))]

for (message_type, num) in message_type_edges:
    print("Message type {} has {} edges".format(message_type, num))

Message type ('user', 'buy', 'product') has 124 edges


In [108]:
data_snap.node_label #edge_index[('user','buy','product')].t().shape # now the index corresponding to each index in each category

{'user': tensor([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]),
 'product': tensor([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1])}

## Data Splitting

In [109]:
from deepsnap.dataset import GraphDataset

In [110]:
task = 'link_pred'
edge_train_mode = 'all'
dataset = GraphDataset([data_snap], task = task,  edge_train_mode=edge_train_mode)

In [111]:
dataset[0].edge_index[('user','buy','product')].shape

torch.Size([2, 124])

In [112]:
datas_train[0].node_feature['user'].shape

torch.Size([61, 1153])

In [113]:
datas_train, datas_valid, dataset_test = dataset.split(transductive=True, split_ratio=[0.8, 0.1, 0.1])

/Users/marceloyou/opt/anaconda3/lib/python3.8/site-packages/deepsnap/hetero_graph.py:3138: UserWarning: __floordiv__ is deprecated, and its behavior will change in a future version of pytorch. It currently rounds toward 0 (like the 'trunc' function NOT 'floor'). This results in incorrect rounding for negative values. To keep the current behavior, use torch.div(a, b, rounding_mode='trunc'), or for actual floor division, use torch.div(a, b, rounding_mode='floor').
  row[message_type] = perm[message_type] // num_nodes[tail_type]


In [116]:
datas_train[0].edge_index[('product','attract','user')].shape

torch.Size([2, 99])

In [117]:
dataset[0].node_types

['user', 'product']

In [154]:
  print("Original graph has {} edges".format(dataset[0].edge_index[('user','buy','product')].shape[1]))
  print()

  print("Train set has {} message passing edge".format(datas_train[0].edge_index[('user','buy','product')].shape[1]))
  print("Train set has {} supervision (positive) edges".format(datas_train[0].edge_label_index[('user','buy','product')].shape[1] // 2))

  print()
  print("Validation set has {} message passing edge".format(datas_valid[0].edge_index[('user','buy','product')].shape[1]))
  print("Validation set has {} supervision (positive) edges".format(datas_valid[0].edge_label_index[('user','buy','product')].shape[1] // 2))

  print()
  print("Test set has {} message passing edge".format(dataset_test[0].edge_index[('user','buy','product')].shape[1]))
  print("Test set has {} supervision (positive) edges".format(dataset_test[0].edge_label_index[('user','buy','product')].shape[1] // 2))

Original graph has 124 edges

Train set has 99 message passing edge
Train set has 99 supervision (positive) edges

Validation set has 99 message passing edge
Validation set has 12 supervision (positive) edges

Test set has 111 message passing edge
Test set has 13 supervision (positive) edges


### Split data set checking

In [154]:
def check_disjoint(edge_index1, edge_index2):
    edge_index1 = edge_index1.detach().numpy()
    edge_index2 = edge_index2.detach().numpy()
    
    intercept = [x for x in set(tuple(x) for x in edge_index1) &
                   set(tuple(x) for x in edge_index2)]
    disjoint = len(intercept) == 0
    return disjoint

num_train_edges = datas_train[0].edge_label_index[('user','buy','product')].shape[1] // 2
train_index1 =  datas_train[0].edge_label_index[('user','buy','product')][:, :num_train_edges]
train_index2 =  datas_train[0].edge_label_index[('user','buy','product')][:num_train_edges]

print('Postive and nagative edges: ', check_disjoint(train_index1, train_index2))

Postive and nagative edges:  True


In normal settings, the negative sample ratio is normally 1.Negative samples will be resampled when batch is called

In [62]:
datasets = {}
follow_batch = []
from torch.utils.data import DataLoader
datasets['train'],datasets['valid'], dataset['test'] = datas_train,datas_valid, dataset_test
dataloaders = {
    split: DataLoader(ds, collate_fn = Batch.collate(follow_batch),
                     batch_size = 1, shuffle = (split == 'train'))
    for split, ds in datasets.items()
}

print("check the negative sampling of two batch")

for batch in dataloaders['train']:
    num_edges = batch.edge_label_index[('user','buy','product')].shape[1] // 2
    neg_edge_1 = batch.edge_label_index[('user','buy','product')][:, num_edges:]
    
for batch in dataloaders['valid']:
    num_edges = batch.edge_label_index[('user','buy','product')].shape[1] // 2
    neg_edge_2 = batch.edge_label_index[('user','buy','product')][:, num_edges:]
    
print('Batch1: \n', neg_edge_1)
print('Batch2: \n', neg_edge_2)
print('Whether Batch1 equal to batch2: ', torch.equal(neg_edge_1, neg_edge_2) )


check the negative sampling of two batch
Batch1: 
 tensor([[ 53307,  11193,  59911,  ...,  54221,  38121,   2011],
        [181166, 111196,  47537,  ...,  44294,  62045,   8673]])
Batch2: 
 tensor([[ 48890,  46336,  30079,  ...,  27309,  36820,  50808],
        [ 46779, 215916,  56737,  ...,  26691,  66502,  70872]])
Whether Batch1 equal to batch2:  False


## Feature Engineering on nodes

In [196]:
print('Before:', dataset[0].node_feature['user'].shape[1])

Before: 1154


In [149]:
dataset2 = dataset
pr = nx.pagerank(data_nx)
node_num = len(list(data_nx.nodes(data = True)))
pr_feature = torch.tensor([pr[node] for node in range(node_num)], dtype = torch.float32).view(node_num, 1)

user_num = dataset[0].node_feature['user'].shape[0]
dataset[0].node_feature['user'] = torch.cat([dataset[0].node_feature['user'], pr_feature[:user_num]], dim = 1)

dataset[0].node_feature['product'] = torch.cat([dataset[0].node_feature['product'], pr_feature[user_num:]], dim = 1)

In [168]:
print('Check Transformation: ')
#print('Before: ', dataset2[0].node_feature['user'].shape)
print('After: ', dataset[0].node_feature['user'].shape)

Check Transformation: 
After:  torch.Size([61, 1154])


2.903447610934537e-06

## Model Building

For the model, when we feed each batch, it has two layers of convolution layer, we select the source and destination embeddings, we multiply and then sum up for our final results. When predict, we feed another sigmoid function

In [179]:
import copy
import torch
import numpy as np
import networkx as nx
import matplotlib.pyplot as plt

from deepsnap.graph import Graph
from deepsnap.batch import Batch
from deepsnap.dataset import GraphDataset
from torch_geometric.datasets import Planetoid, TUDataset

from torch.utils.data import DataLoader

import torch.nn.functional as F
from torch_geometric.nn import SAGEConv
import torch.nn as nn

import copy
import torch
import deepsnap
import numpy as np
import torch.nn as nn
import torch.nn.functional as F
import torch_geometric.nn as pyg_nn

from sklearn.metrics import f1_score
from deepsnap.hetero_gnn import forward_op
from deepsnap.hetero_graph import HeteroGraph
from torch_sparse import SparseTensor, matmul

from deepsnap.hetero_gnn import (
    HeteroSAGEConv,
    HeteroConv,
    forward_op
)
from copy import deepcopy

### HeteroGNNConv Ideas

The heterogenos graph has twp types of node user and product, the message type is (u, buy, product). Since there is only one type of link, there is only one kind of heterogenous message layers. 

In [29]:
def generate_2convs_link_pred_layers(hete, conv, hidden_size):
    convs1 = {}
    convs2 = {}
    for message_type in hete.message_types:
        s_type = message_type[0]
        d_type = message_type[2]
        s_feed_dim = hete.num_node_features(s_type)
        d_feed_dim = hete.num_node_features(d_type)
        convs1[message_type] = conv(s_feed_dim, hidden_size, d_feed_dim)
        convs2[message_type] = conv(hidden_size,hidden_size,hidden_size)
    return convs1, convs2
    

In [164]:
datas_train[0].message_types

[('user', 'buy', 'product'), ('product', 'attract', 'user')]

In [188]:
class LinkPrediction(nn.Module):
    def __init__(self,args, conv1, conv2, hetero):
        super(LinkPrediction, self).__init__()
        self.agrs = args
        self.convs1 = HeteroConv(conv1)
        self.convs2 = HeteroConv(conv2)
        self.loss_fn = nn.BCEWithLogitsLoss()
        self.bns1 = nn.ModuleDict()
        self.bns2 = nn.ModuleDict()
        self.relus1 = nn.ModuleDict()
        self.relus2 = nn.ModuleDict()
        self.post_mps = nn.ModuleDict()
        
        for node_type in hetero.node_types:
            self.bns1[node_type] = nn.BatchNorm1d(args['hidden_size'])
            self.bns2[node_type] = nn.BatchNorm1d(args['hidden_size'])
            self.relus1[node_type] = nn.LeakyReLU()
            self.relus2[node_type] = nn.LeakyReLU()
            
        
    def forward(self, data):
        x = data.node_feature
        #print(x)
        edge_index = data.edge_index
        x = self.convs1(x, edge_index)
        #print(x)
        x = forward_op(x, self.bns1)
        x = forward_op(x, self.relus1)
        #print(x)
        x = self.convs2(x, edge_index)
        x = forward_op(x, self.bns2)
        pred = {}
        
        for message_type in list(data.edge_index.keys()):
            if message_type == ('user', 'buy', 'product'):
                nodes_first = torch.index_select(x['user'], 0, data.edge_label_index[message_type][0,:].long())
                nodes_second = torch.index_select(x['product'], 0, data.edge_label_index[message_type][1,:].long())

                pred[message_type] = torch.sum(nodes_first * nodes_second, dim = -1)
            else:
                pass
        return pred
    
    def loss(self, pred, y):
        loss = 0
        for key in pred:
    
             p = torch.sigmoid(pred[key])
             #print(p, pred[key])
             y = torch.tensor([i.item() - 1 if i.item() == 2 else i.item() for i in y[key] ])
 
             loss += self.loss_fn(p, y.type(pred[key].dtype))
        return loss
    

    
conv1, conv2 = generate_2convs_link_pred_layers(dataset[0], HeteroSAGEConv, 256)
    
        
        


In [189]:
def train(model, dataloaders, optimizer, args):
    val_max = 0
    best_moedl = model
    t_accu, v_accu, e_accu = [], [], []
    losses = []
    for epoch in range(1, args['epoches'] + 1):
        for iter_i, batch in enumerate(dataloaders['train']):
            model.train()
            optimizer.zero_grad()
            pred = model(batch)
            loss = model.loss(pred, batch.edge_label)
            loss.backward()
            optimizer.step()
            losses.append(loss)
            log = 'Epoch: {:03d}, Train loss: {:.4f}ff}'
            accs = test(model, dataloaders, args)
            t_accu.append(accs['train'])
            v_accu.append(accs['valid'])
            e_accu.append(accs['test'])
            
            print(log.format(epoch, loss.item(), accs['train'],accs['valid'],accs['test']))
            if val_max < accs['valid']:
                val_max = accs['valid']
                best_model = deepcopy(model)
    log = 'Best: Train: {:.4f}, Val: {:.4f}, Test: {:.4f}'
    accs = test(best_model, dataloaders, args)
    print(log.format(accs['train'], accs['valid'], accs['test']))
        
    return t_accu, v_accu, e_accu, best_model

In [190]:
def test(model, dataloaders, args):
    model.eval()
    
    accs = {}
    
    for mode, dataloader in dataloaders.items():
        acc = 0
        for i, batch in enumerate(dataloader):
            num = 0
            pred = model(batch)
            
            for key in pred:
                p = torch.sigmoid(pred[key]).detach().numpy()
                pred_label = np.zeros_like(p, dtype = np.float64)
                pred_label[np.where(p > 0.5)[0]] = 1
                pred_label[np.where(p < 0.5)[0]] = 0
                acc += np.sum(pred_label == batch.edge_label[key].numpy())
                num += len(pred_label)
        accs[mode] = acc / num
    return accs

                

In [197]:
import time
def main():
    
    args = {'epoches': 200,
           'hidden_size': 128,
            'path':'/Users/marceloyou/Desktop/算法/RecommanationSystem/GNN(GCN)/src/',
           'output_path': time.strftime('checkpoints/model_' + '%m%d_%H:%M:%S.pth') }
    
    train_loader = DataLoader(datas_train, collate_fn = Batch.collate(), batch_size = 1)
    valid_loader = DataLoader(datas_valid, collate_fn = Batch.collate(), batch_size = 1)
    test_loader = DataLoader(datas_train, collate_fn = Batch.collate(), batch_size = 1)
    
    dataloaders = {'train': train_loader, 'valid':valid_loader, 'test':test_loader}
    hidden_size = args['hidden_size']
    
    conv1, conv2 = generate_2convs_link_pred_layers(dataset[0], HeteroSAGEConv, hidden_size)
    model = LinkPrediction(args, conv1, conv2, dataset[0])    
    optimizer = torch.optim.Adam(model.parameters(), lr = 0.001, weight_decay = 5e-4)
    t_accu, v_accu, e_accu, best_model = train(model, dataloaders, optimizer, args)
    
    model = {'model':model.state_dict()}
    torch.save(model, os.path.join(args['path'], args['output_path']))

In [199]:
main()

Epoch: 001, Train loss: 0.7846, Train: 0.4293, Val: 0.1667, Test: 0.4293
Epoch: 002, Train loss: 0.6529, Train: 0.4394, Val: 0.3333, Test: 0.4394
Epoch: 003, Train loss: 0.6117, Train: 0.4646, Val: 0.3333, Test: 0.4646
Epoch: 004, Train loss: 0.6007, Train: 0.4798, Val: 0.3333, Test: 0.4798
Epoch: 005, Train loss: 0.6199, Train: 0.4798, Val: 0.3333, Test: 0.4798
Epoch: 006, Train loss: 0.6096, Train: 0.4747, Val: 0.3750, Test: 0.4747
Epoch: 007, Train loss: 0.6236, Train: 0.4747, Val: 0.3750, Test: 0.4747
Epoch: 008, Train loss: 0.6116, Train: 0.4899, Val: 0.3750, Test: 0.4899
Epoch: 009, Train loss: 0.6221, Train: 0.4899, Val: 0.3333, Test: 0.4899
Epoch: 010, Train loss: 0.6016, Train: 0.4899, Val: 0.2917, Test: 0.4899
Epoch: 011, Train loss: 0.6148, Train: 0.4899, Val: 0.2500, Test: 0.4899
Epoch: 012, Train loss: 0.6065, Train: 0.4899, Val: 0.2500, Test: 0.4899
Epoch: 013, Train loss: 0.6046, Train: 0.4899, Val: 0.2500, Test: 0.4899
Epoch: 014, Train loss: 0.5918, Train: 0.4899, Val:

In [133]:
train_loader = DataLoader(datas_train, collate_fn = Batch.collate(), batch_size = 1)
for batch in train_loader:
    print(list(batch.edge_index.keys()))
    print(batch.edge_label_index)

[('user', 'buy', 'product'), ('product', 'attract', 'user')]
{('user', 'buy', 'product'): tensor([[ 48,   2,  57,  57,  39,  50,  21,  38,  16,  17,  40,  57,  14,  35,
           0,  28,  56,  29,  11,  18,  48,  26,  30,  28,  12,  47,   9,  42,
           2,  25,  27,  14,  48,  48,  48,   1,  23,  35,  18,  36,  10,  14,
           1,  38,  31,  28,  60,  48,   6,  38,  48,  40,  48,  58,   1,  30,
           2,  17,  35,  36,  48,   6,  51,  15,   2,   5,  12,  35,  57,  35,
          22,  59,  34,  10,   8,   4,  32,  13,  46,  18,   2,   7,  13,  54,
          28,  38,  38,  49,  57,  10,  43,   6,  24,  20,  15,   6,  45,  55,
           1,  40,   9,   1,  20,  12,  12,   8,  27,  18,   2,   0,   3,  53,
           0,  51,  52,   0,   7,  12,  40,   0,  25,  16,  11,  27,  18,  33,
          52,  38,  39,  39,  34,  47,  51,  35,  17,  58,  10,   6,  39,   7,
          41,  35,  44,  48,  41,  16,  54,   5,   1,  34,  19,  41,  17,  56,
           4,   0,  17,  14,  39,  17,  5

In [169]:
datas_train[0].edge_label

{('user',
  'buy',
  'product'): tensor([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
         2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
         2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
         2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
         2, 2, 2, 2, 2, 2]),
 ('product',
  'attract',
  'user'): tensor([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 

99